In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing dependencies
import numpy as np
import string
import random
import re
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.models import Sequential
from numpy import array, argmax, random, take
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RNN, SimpleRNN, LSTM,  Embedding, RepeatVector
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
%matplotlib inline
#For plotting the matplotlib graphs in notebook

In [ ]:
Data_path="/content/drive/MyDrive/Datasets/3Gram_love_data.txt"

In [ ]:
import pandas as pd
column_names = ['word1', 'word2', 'word3']

input_3gram = pd.read_csv(Data_path, delimiter='\t', names=column_names) #Importing csv file with column names
print("shape of data", input_3gram.shape)

shape of data (5351, 3)


In [ ]:
print("Few sample records from data \n", input_3gram.sample(10))

Few sample records from data 
      word1 word2    word3
1133  love    of      the
4123  love    it     when
3580  love    to      see
1942  love   and  support
2935  love    to      see
3929  love    it     when
3747  love    it     when
4185  love    it     when
392   hate    to      use
287   hate    to     have


In [ ]:
print("\nFrequency of word1 values \n", input_3gram["word1"].value_counts())
print("\nFrequency of word2 values \n", input_3gram["word2"].value_counts())


Frequency of word1 values 
 love      4327
loved      416
hate       400
hated       80
loves       72
lovely      24
loving      24
hates        8
Name: word1, dtype: int64

Frequency of word2 values 
 to         1866
it         1361
the         548
with        240
him         144
you         144
of          136
her         104
for          96
and          88
what         56
is           48
each         40
in           40
ones         32
me           32
nothing      32
them         32
as           24
every        24
more         16
going        16
that         16
being        16
affair       16
my           16
about         8
your          8
on            8
letter        8
most          8
thy           8
view          8
song          8
makes         8
got           8
this          8
at            8
a             8
one           8
hearing       8
story         8
all           8
lost          8
man           8
when          8
husband       8
Name: word2, dtype: int64


In [ ]:

unique_words = []
for i in list(input_3gram.columns.values):
    for j in pd.unique(input_3gram[i]):
        unique_words.append(j)
unique_words = np.unique(unique_words)


print('Count of unique words overall:', len(unique_words))
print('unique words list:', unique_words)

Count of unique words overall: 139
unique words list: ['a' 'able' 'about' 'admit' 'affair' 'affection' 'all' 'and' 'another'
 'answer' 'as' 'at' 'be' 'because' 'being' 'better' 'between' 'bother'
 'break' 'care' 'cared' 'come' 'concern' 'country' 'cut' 'disappoint' 'do'
 'each' 'every' 'fact' 'feel' 'feeling' 'find' 'first' 'for' 'from' 'get'
 'go' 'god' 'going' 'got' 'hate' 'hated' 'hates' 'have' 'he' 'hear'
 'hearing' 'her' 'here' 'him' 'his' 'husband' 'i' 'idea' 'if' 'in'
 'interrupt' 'is' 'it' 'kind' 'know' 'leave' 'letter' 'life' 'like'
 'listen' 'look' 'lost' 'lot' 'love' 'loved' 'lovely' 'loves' 'loving'
 'make' 'makes' 'man' 'marriage' 'me' 'minute' 'more' 'most' 'much'
 'music' 'my' 'nature' 'neighbor' 'not' 'nothing' 'of' 'on' 'one' 'ones'
 'or' 'other' 'over' 'play' 'respect' 'say' 'see' 'sit' 'smell' 'so'
 'someone' 'song' 'sound' 'story' 'stronger' 'support' 'take' 'talk'
 'tell' 'than' 'that' 'the' 'them' 'they' 'think' 'this' 'thought' 'thy'
 'to' 'too' 'united' 'use' 'v

In [ ]:
word_indices = dict((w, i) for i, w in enumerate(unique_words))
indices_words = dict((i, w) for i, w in enumerate(unique_words))

In [ ]:
### Onehot encoding of word1
word1 = input_3gram['word1'].map(word_indices)
word1_onehot = keras.utils.to_categorical(np.array(word1), num_classes=len(word_indices))
print("word1_onehot shape is ",word1_onehot.shape)

word1_onehot shape is  (5351, 139)


In [ ]:
##one hot encoding for word2 and word3 
word2 = input_3gram['word2'].map(word_indices)
word2_onehot = keras.utils.to_categorical(np.array(word2), num_classes=len(word_indices))
print("word2_onehot shape is ",word2_onehot.shape)

word3 = input_3gram['word3'].map(word_indices)
word3_onehot = keras.utils.to_categorical(np.array(word3), num_classes=len(word_indices))
print("word3_onehot shape is ",word3_onehot.shape)

word2_onehot shape is  (5351, 139)
word3_onehot shape is  (5351, 139)


## RNNs

## Word prediction using RNN model

In [ ]:
word1_word2 = input_3gram[['word1','word2']]
for i in list(word1_word2.columns.values):
    word1_word2[i] = word1_word2[i].map(word_indices)

word1_word2=np.array(word1_word2)
#The same data is reshaped with similar structure but appended with 1 value to make it 3d array
word1_word2=np.reshape(word1_word2,(word1_word2.shape[0],2,1))
word1_word2_onehot = keras.utils.to_categorical(np.array(word1_word2), num_classes=len(word_indices))
print("word1_word2_onehot shape", word1_word2_onehot.shape)

word1_word2_onehot shape (5351, 2, 139)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print("time steps" , word1_word2_onehot.shape[1])
print("Input nodes" , word1_word2_onehot.shape[2])
print("output nodes" , word3_onehot.shape[1])

time steps 2
Input nodes 139
output nodes 139


In [ ]:
model_rnn = Sequential()
#model.add(SimpleRNN('number of hidden nodes in each rnn cell', input_shape=(timesteps, input_data_dim)))
model_rnn.add(SimpleRNN(30, input_shape=(word1_word2_onehot.shape[1],word1_word2_onehot.shape[2]))) 
model_rnn.add(Dense(word3_onehot.shape[1], activation='softmax'))
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 30)                5100      
                                                                 
 dense_1 (Dense)             (None, 139)               4309      
                                                                 
Total params: 9,409
Trainable params: 9,409
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile network
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_rnn.fit(word1_word2_onehot, word3_onehot, epochs=20)

Epoch 1/20
168/168 [==============================] - 3s 4ms/step - loss: 3.7305 - accuracy: 0.3898
Epoch 2/20
168/168 [==============================] - 1s 4ms/step - loss: 2.7377 - accuracy: 0.4971
Epoch 3/20
168/168 [==============================] - 1s 4ms/step - loss: 2.3265 - accuracy: 0.5317
Epoch 4/20
168/168 [==============================] - 1s 4ms/step - loss: 2.0590 - accuracy: 0.5726
Epoch 5/20
168/168 [==============================] - 1s 4ms/step - loss: 1.8739 - accuracy: 0.5855
Epoch 6/20
168/168 [==============================] - 1s 4ms/step - loss: 1.7350 - accuracy: 0.6064
Epoch 7/20
168/168 [==============================] - 1s 4ms/step - loss: 1.6272 - accuracy: 0.6247
Epoch 8/20
168/168 [==============================] - 1s 4ms/step - loss: 1.5374 - accuracy: 0.6365
Epoch 9/20
168/168 [==============================] - 1s 4ms/step - loss: 1.4648 - accuracy: 0.6485
Epoch 10/20
168/168 [==============================] - 1s 4ms/step - loss: 1.4055 - accuracy: 0.6571

In [ ]:
def rnn_word_pred(in_text):
    print("Input is - " , in_text, end = ' ')
    encoded = [word_indices[i] for i in in_text]
    encoded = np.array(encoded).reshape(1,2,1)
    encoded =keras.utils.to_categorical(np.array(encoded), num_classes=len(word_indices))
    #ypred = model_rnn.predict_classes(encoded, verbose=0)[0]
    predict_(x=model_rnn.predictencoded) 
    classes_x=np.argmax(predict_x, axis=1)
   # print(classes_x)
    print(indices_words[classes_x[0]])

In [ ]:
rnn_word_pred(['love', 'it'])
rnn_word_pred(['love', 'to'])
rnn_word_pred(['love', 'the'])

Input is -  ['love', 'it'] when
Input is -  ['love', 'to'] see
Input is -  ['love', 'the'] way
